In [34]:
# Homework 2 Part 2 (due 7/07/2024)

# Health-care assistance via probabilistic graphical modeling

### Objective
In this project, you will create a health-care assistance bot that can suggest diagnoses for a set of symptoms based on a probabilistic graphical model.

### Step 1: Review 
Review the code from the Bayesian networks exercise.

### Step 2: Acquire data
View this [research article](https://www.nature.com/articles/ncomms5212) and download its supplementary data sets 1, 2 and 3. These data sets include the occurrences of diseases, symptoms, and their co-occurrences in the scientific literature. (For the purpose of this exercise, we are going to assume that the frequency of co-occurrences of diseases and symptoms in scientific papers is proportional to the co-occurence frequencies of actual disease cases and symptoms.)

### Step 3: Create a Bayesian network
Using commands from the `pgmpy` library, create a Bayesian network in which the probability of exhibiting a symptom is conditional on the probability of having an associated disease. 

### Step 4: Initialize priors
Use the disease occurrence data to assign prior probabilities for diseases.

### Step 5: Calculate conditional probability tables
Use the co-occurrence data to define CPTs for each connected pair of disease and symptoms. (Hint: You may need to assign some occurrences of symptoms to an "idiopathic disease" to create valid CPTs.)

### Step 6:
Create a minimal interface in which your bot asks a users for a list of observed symptoms and then returns the name of the disease that is the most likely match to the symptoms. (Hint: Review the input/output commands that you have used in last week's homework.)

#### Step 2

Let's load the data to get an idea of what it looks like.

In [35]:
import pandas as pd
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [36]:
diseases_df = pd.read_csv("supmat1.txt", sep='\t')
diseases_df.head()

,MeSH Disease Term,PubMed occurrence
0,Breast Neoplasms,122226
1,Hypertension,107294
2,Coronary Artery Disease,82819
3,Lung Neoplasms,78009
4,Myocardial Infarction,75945


In [37]:
symptoms_df = pd.read_csv("supmat2.txt", sep='\t')
symptoms_df.head()

,MeSH Symptom Term,PubMed occurrence
0,Body Weight,147857
1,Pain,103168
2,Obesity,100301
3,Anoxia,47351
4,Mental Retardation,43883


In [38]:
cooccurrences_df = pd.read_csv("supmat3.txt", sep='\t')
cooccurrences_df.head()

,MeSH Symptom Term,MeSH Disease Term,PubMed occurrence,TFIDF score
0,"Aging, Premature",Respiratory Syncytial Virus Infections,1,3.464551
1,"Aging, Premature",Orthomyxoviridae Infections,1,3.464551
2,"Aging, Premature",HIV Infections,3,10.393654
3,"Aging, Premature",Acquired Immunodeficiency Syndrome,3,10.393654
4,"Aging, Premature",Breast Neoplasms,1,3.464551


##### Before we even start with building the network we must adjust to our RAM capabilities. Let us take the first 400 co-occurrences and accordingly cut diseases and symptoms dataframes.

Note that because of that we won't have to worry about idiopathic diseases for symptoms that don't match a known disease.

In [39]:
cooccurrences_df = cooccurrences_df.head(400)
# Get unique diseases and symptoms from the first 400 rows of cooccurrences_df
unique_diseases = cooccurrences_df['MeSH Disease Term'].unique()
unique_symptoms = cooccurrences_df['MeSH Symptom Term'].unique()

# Filter diseases_df and symptoms_df
diseases_df = diseases_df[diseases_df['MeSH Disease Term'].isin(unique_diseases)]
symptoms_df = symptoms_df[symptoms_df['MeSH Symptom Term'].isin(unique_symptoms)]

#### Step 3

In [40]:
# Initialize Bayesian Network
# The edges go from diseases to symptoms, and, thankfully, we have the edgelist in the cooccurrences dataset
ds_edgelist = [(cooccurrences_df.iloc[i, 1], cooccurrences_df.iloc[i, 0]) for i in range(len(cooccurrences_df))]
bayesian_net = BayesianNetwork(ds_edgelist)

#### Step 4

In [41]:
# Assign Prior Probabilities based on occurrence data of diseases
# We will treat occurrences in scientific papers as the ground truth for real life and count the occurrence of each disease proportional to all occurrences.
occurrences = diseases_df.iloc[:, 1]
total_occurrences = occurrences.sum()
diseases_df['prior_probabilities'] = occurrences/total_occurrences
diseases_df.head()

,MeSH Disease Term,PubMed occurrence,prior_probabilities
0,Breast Neoplasms,122226,0.031477
1,Hypertension,107294,0.027631
2,Coronary Artery Disease,82819,0.021328
3,Lung Neoplasms,78009,0.020089
5,HIV Infections,66601,0.017152


#### Step 5

In [43]:
# Calculate total co-occurrences for each disease
# Remember that symptoms are conditional only on their parent diseases
total_cooccurrences = cooccurrences_df.groupby('MeSH Disease Term')['PubMed occurrence'].sum().reset_index()
total_cooccurrences.columns = ['MeSH Disease Term', 'total_cooccurrences']

# Merge total co-occurrences with the co-occurrences DataFrame
cooccurrences_df = cooccurrences_df.merge(total_cooccurrences, on='MeSH Disease Term')

# Calculate CPTs
cooccurrences_df['CPT'] = cooccurrences_df['PubMed occurrence'] / cooccurrences_df['total_cooccurrences']

print(cooccurrences_df)

    MeSH Symptom Term                       MeSH Disease Term  \
0    Aging, Premature  Respiratory Syncytial Virus Infections   
1    Aging, Premature             Orthomyxoviridae Infections   
2    Aging, Premature                          HIV Infections   
3    Aging, Premature      Acquired Immunodeficiency Syndrome   
4    Aging, Premature                        Breast Neoplasms   
..                ...                                     ...   
395          Asthenia                       Epilepsy, Absence   
396          Asthenia                      Migraine Disorders   
397          Asthenia                           Hydrocephalus   
398          Asthenia                    Empty Sella Syndrome   
399          Asthenia                         Hypopituitarism   

     PubMed occurrence  TFIDF score  total_cooccurrences       CPT  
0                    1     3.464551                    1  1.000000  
1                    1     3.464551                    1  1.000000  
2           

In [44]:
# Iterate over unique diseases in cooccurrences_df
cpds = []
for disease in cooccurrences_df['MeSH Disease Term'].unique():
    # Filter cooccurrences_df for the current disease
    symptom_cpt = cooccurrences_df[cooccurrences_df['MeSH Disease Term'] == disease]
    
    # Create CPT values for symptoms given the disease
    values = symptom_cpt['PubMed occurrence'] / symptom_cpt['PubMed occurrence'].sum()
    
    # Get the list of symptoms for this disease
    symptoms_for_disease = symptom_cpt['MeSH Symptom Term'].unique()
    
    # Create a CPD for each symptom given the disease
    for symptom in symptoms_for_disease:
        # Extract current probability of the symptom given the disease
        current_probability = values[symptom_cpt['MeSH Symptom Term'] == symptom].values[0]
        
        # Calculate complement probability
        complement_probability = 1 - current_probability
        
        # Calculate probability of symptom without disease (baseline probability)
        baseline_probability = symptoms_df.loc[symptoms_df['MeSH Symptom Term'] == symptom, 'PubMed occurrence'].values[0] / total_occurrences
        
        # Calculate complement of baseline probability
        complement_baseline_probability = 1 - baseline_probability
        
        # Create the 2x2 array for TabularCPD values
        symptom_values = np.array([[current_probability, baseline_probability],
                                   [complement_probability, complement_baseline_probability]])
        
        # Create TabularCPD and add it to cpds list
        cpd = TabularCPD(variable=symptom, variable_card=2,
                         values=symptom_values,
                         evidence=[disease], evidence_card=[2])
        cpds.append(cpd)


# Add CPDs to the Bayesian Network
bayesian_net.add_cpds(*cpds)

#### Step 6

Let's copy the workflow for input in the example code from Homework one and build a class that checks symptoms. It must be created given a proper Bayesian Network.

Note that because we have limited the network inputs to the first 400 cooccurrences, we only have 2 possible symptoms: 'Asthenia', and 'Aging, Premature'. :D

In [ ]:
# Assuming you have already created and populated your Bayesian Network (`bayesian_net`) and loaded data

# Function to interact with the user and return the most likely disease
def symptom_checker(bayesian_net):
    # Prompt user for symptoms, comma-separated
    user_input = input("Please enter observed symptoms separated by commas: ")
    observed_symptoms = [sym.strip() for sym in user_input.split(',')]

    # Initialize an empty evidence dictionary
    evidence = {}

    # Match observed symptoms with available symptoms in the network
    available_symptoms = bayesian_net.nodes()

    for symptom in observed_symptoms:
        if symptom in available_symptoms:
            evidence[symptom] = 1  # Assuming presence of symptom (state 1 in binary CPD)

    # Perform variable elimination to find the most probable disease
    infer = VariableElimination(bayesian_net)
    disease_probabilities = infer.query(variables=['MeSH Disease Term'], evidence=evidence)

    # Extract the disease with the highest probability
    max_prob_disease = disease_probabilities['MeSH Disease Term'].values.argmax()

    return max_prob_disease

# Let's ask for input!
if __name__ == "__main__":
    disease_match = symptom_checker(bayesian_net)
    print(f"The most likely disease match based on observed symptoms is: {disease_match}")

It is a bit silly given that we don't have enough data and can't load a proper Bayesian Network. Thus, we have only two symptoms and plenty of diseases which makes probabilities all wacky. Especially since we computed priors on the truncated data so that CPDs load properly, and all probability computations make sense and add up to 1 and such.